# Setup

### Imports

In [3]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

Load the TensorBoard notebook extension

%load_ext tensorboard

### Constants

In [27]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

## Vectorize an example sentence

Tokenize the sentence:

In [28]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
# print(len(tokens))

Create a vocabulary to save mappings from tokens to integer indices.

In [18]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


Create an inverse vocabulary to save mappings from integer indices to tokens.

In [7]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


Vectorize your sentence.

In [8]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


## Generate skip-grams from one sentence

In [19]:
'''
    TODO: 2021-10-02 - Mover isso pra definicao de constantes
'''
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)

# print(len(positive_skip_grams))
print(positive_skip_grams)

# for target, context in positive_skip_grams[:5]:
#   print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

[[3, 2], [1, 4], [6, 7], [2, 1], [3, 4], [1, 5], [5, 6], [5, 1], [4, 2], [7, 6], [4, 5], [4, 3], [6, 5], [1, 6], [2, 4], [5, 4], [7, 1], [6, 1], [3, 1], [4, 1], [1, 2], [5, 3], [1, 7], [1, 3], [3, 5], [2, 3]]


## Negative sampling for one skip-gram
- Sample `num_ns` number of negative samples for a given target word in a window;
- You can call the function on one skip-grams's target word and pass the context word as true class to exclude it from being sampled;

In [68]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]
# print(positive_skip_grams[0])

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
print(context_class)

negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class, # class that should be sampled as 'positive'
    num_true=1,                 # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,         # number of negative context words to sample
    unique=True,                # all the negative samples should be unique
    range_max=vocab_size,       # pick index of the samples from [0, vocab_size]
    seed=SEED,                     # seed for reproducibility
    name="negative_sampling"    # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([[2]], shape=(1, 1), dtype=int64)
tf.Tensor([0 2 3 1], shape=(4,), dtype=int64)
['<pad>', 'wide', 'road', 'the']


## Construct one training example
    A tuple of (target, context, label) tensors constitutes one training example
_In (skip-gram negative sampling Word2Vec model)_


- Batch 1 positive `context_word` and `num_ns` negative context words into one tensor;
- This produces a set of positive skip-grams _(labelled as 1)_ & negative samples _(labelled as 0)_ for each target word;
- Notice that the target is of shape (1,) while the context and label are of shape (1+num_ns,)

In [73]:
# Add a dimension so you can use concatenation (on the next step).
# negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)
# print(negative_sampling_candidates)

# print(tf.expand_dims(negative_sampling_candidates, 1))
# print(tf.expand_dims(context_class, 1))

# Concat positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)
# print(context)

# Label first context word as 1 (positive) followed by num_ns 0s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape target to shape (1,) and context and label to (num_ns+1,).
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

print("target  :", target)
print("context :", context)
print("label   :", label)

target_index    : 3
target_word     : road
context_indices : [2 0 2 3 1]
context_words   : ['wide', '<pad>', 'wide', 'road', 'the']
label           : [1 0 0 0 0]
target  : tf.Tensor(3, shape=(), dtype=int32)
context : tf.Tensor([2 0 2 3 1], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
